In [ ]:
from dkube.sdk import *
import os

## Initializing feilds

In [ ]:
user = os.getenv('USERNAME')
token = os.getenv('DKUBE_USER_ACCESS_TOKEN')
inp_path = "/opt/dkube/input"
out_path = "/opt/dkube/output"
code_name = "mnist-fs"
dataset_name = "mnist"
training_fs = "mnist-fs"
model_name = "mnist"
framework = "tensorflow_1.14"
image = "ocdr/d3-datascience-tf-cpu:v1.14-3"
preprocess_script = f"python featureset.py --fs {training_fs}"
training_script = f"python model.py --fs {training_fs}"
transformer_script = "tf/classification/mnist-fs/digits/transformer/transformer.py"

In [ ]:
# Dkube client handle
api = DkubeApi(token=token)
api.wait_interval = 20

## Creating Repos

In [ ]:
print(f"Adding code {code_name}")
code = DkubeCode(user, name=code_name)
code.update_git_details("https://github.com/oneconvergence/dkubeio-examples/tree/master/tf/classification/mnist-fs/digits/classifier/program")
try:
    api.create_code(code)
    print(f"Code {code_name} added")
except BaseException as e:
    print(e)

print(f"Adding dataset {dataset_name}")
dataset = DkubeDataset(user, name=dataset_name)
dataset.update_dataset_source(source='git')
dataset.update_git_details("https://github.com/oneconvergence/dkubeio-examples-data/tree/master/tf/mnist")
try:
    api.create_dataset(dataset)
    print(f"Dataset {dataset_name} added")
except BaseException as e:
    print(e)

print(f"Adding featureset {training_fs}")
featureset = DkubeFeatureSet(name=training_fs)
try:
    api.create_featureset(featureset)
    print(f"Featureset {training_fs} added")
except BaseException as e:
    print(e)
    
print(f"Adding model {model_name}")
model = DkubeModel(user, name=model_name)
model.update_model_source(source='dvs')
try:
    api.create_model(model)
    print(f"Model {model_name} added")
except BaseException as e:
    print(e)

## Defining and Creating preprocessing run

In [ ]:
preprocessing_name= generate(code_name)
preprocessing = DkubePreprocessing(user, name=preprocessing_name, description='triggered from dkube sdk')
preprocessing.update_container(image_url=image)
preprocessing.update_startupscript(preprocess_script)
preprocessing.add_code(code_name)
preprocessing.add_input_dataset(dataset_name, mountpath=inp_path)
preprocessing.add_output_featureset(training_fs, mountpath=out_path)

In [ ]:
api.create_preprocessing_run(preprocessing)

## Defining and Creating training run

In [ ]:
training_name= generate(code_name)
training = DkubeTraining(user, name=training_name, description='triggered from dkube sdk')
training.update_container(framework=framework, image_url=image)
training.update_startupscript(training_script)
training.add_code(code_name)
training.add_input_featureset(training_fs, mountpath=inp_path)
training.add_output_model(model_name, mountpath=out_path)

In [ ]:
api.create_training_run(training)

## Fetching head version of model

In [ ]:
for each_datum in api.list_models(user = user)[0]["datums"]:
    if each_datum["datum"]["name"] == model_name:
        head_version = each_datum["datum"]["generated"]["head_version"]
        break

In [ ]:
head_version

## Defining and Creating inference from model head version

In [ ]:
serving_name=generate(code_name)
serving = DkubeServing(user, name=serving_name, description='serving deployed from dkube sdk')
serving.set_transformer(transformer=True, script=transformer_script)
serving.update_transformer_code(code=code_name)
serving.update_transformer_image(image_url=image)
serving.update_serving_model(model_name, version = head_version)
serving.update_serving_image(image_url='ocdr/tensorflowserver:1.14')
serving.set_production_deploy() #For production deployment, when MODEL_CATALOG_ENABLED=false
serving.update_autoscaling_config(min_replicas=1, max_concurrent_requests=1)

In [ ]:
api.create_test_inference(serving)